# Reflectomerty tutorial
Prettry straight forward not much comments here

In [1]:
import qcodes as qc
from qcodes.tests.instrument_mocks import DummyInstrument
import random
from qcodes.utils.validators import Numbers
import pandas as pd
import numpy as np
from time import strftime
#import matplotlib

# Dummy instruments below
Lock in getter returns random number

In [2]:
hm=DummyInstrument(name="hameg", gates=["ch1"])

gs=DummyInstrument(name="yoko", gates=["volt"])
sr=DummyInstrument(name="lockin", gates=["R"])
setup=qc.Station(hm,gs,sr) #we can store the whole setup like this...fancy

sr.R.get =   lambda: random.randint(0, 100) #the getter for lock-in returns a random number

a=0 #variable for futuresetter and getters
#here we set up the freq setting and getting
def getter():
    return a

def setterf(x):
    global a
    a=x
    #print('Setting F to {}'.format(x))
    return None

def setterv(x):
    global a
    a=x
    #print('Setting V to {}'.format(x))
    return None

hm.add_parameter("frequency",
                label="Frequency",
                get_cmd=getter,
                set_cmd=setterf,
                unit="Hz",
                vals=Numbers(min_value=1, max_value=1e9))

gs.add_parameter('voltage',
                label='Voltage',
                unit='V',
                set_cmd=setterv,
                get_cmd=getter)

# Actual measurement below
Copied and pasted from real measuring code

# 2D sweep

In [ ]:
#Starting voltage in volts
start_volt=-5
#End voltage in volts
end_volt=5
#Step size in volts
delta_volt=1
#Delay after each step in seconds
delay_volt=1

#Starting frequency in hertz
start_freq=10e6
#End frequency in hertz
end_freq=30e6
#Step size in hertz
delta_freq=10e6
#Delay after each step in seconds
delay_freq=0.1

#Saving the time of measurement we will use this data in the next cell
date=strftime("%y_%m_%d_%H_%M_%S")
#This is the measuring loop with the parameters above
loop=qc.Loop(gs.voltage.sweep(start_volt,end_volt,delta_volt),delay=delay_volt).loop(hm.frequency.sweep(start_freq,end_freq,delta_freq),delay=delay_freq).each(sr.R)
#The results are stored in DataSet named 'data'
data=loop.get_data_set(name="2D reflectometry sweep")

#Plotting part
plot=qc.QtPlot(data.lockin_R)
#plot.add(data.yoko_voltage_set)
#plot.add(data.lockin_R)
plot.autorange()
loop.with_bg_task(plot.update)
loop.run()

# Saving the 2D sweep as csv
We create numpy arrays from the DataSet arrays, then we build a pandas.DataFrame from them. Then we save the DataFrame with a name including the date and time when the measurement was started. The path is set by the value of 'filepath'. The savefile will have a name like: " 'filepath'/2D_yy_mm_dd_hh_mm_ss.csv".

In [ ]:
#this is the dedicated variable for filepath
filepath=""
#first we convert the arrays into numpy arrays one by one
hameg_data=np.ravel(np.array(data.hameg_frequency_set))
lockin_data=np.ravel(np.array(data.lockin_R))
yoko_data=np.repeat(np.array(data.yoko_voltage_set),hameg_data.shape[0]/data.yoko_voltage_set.shape[0])
#then we use pandas to save the csv
df=pd.DataFrame(data=[yoko_data,hameg_data,lockin_data]).T
df.rename(columns={0:"Gate voltage (V)",1:"Frequency (Hz)", 2:"Reflected voltage (V)"},inplace=True)
df.to_csv(filepath+"2D_"+date+".csv",sep=",")
#print(df)
print("All data saved as csv!")

# Saving the 2D sweep as .png
Using the default qcodes.pyqtgraph.save() method.

In [ ]:
plot.save(filepath+"2D_"+date+".png")

## 1D sweep with fixed back gate voltage

In [ ]:
#Fixed back gate voltage of Yokogawa
fixed_volt=1
gs.voltage=fixed_volt

#Starting frequency in hertz
start=10e6
#End frequency in hertz
end=200e6
#Step size in hertz
delta=10e6
#Delay after each step in seconds
delay=1

#storing date and time
date_v=strftime("%y_%m_%d_%H_%M_%S")

#This is the measuring loop with the parameters above
loop_v=qc.Loop(hm.frequency.sweep(start,end,delta),delay=delay).each(sr.R)
#The results are stored in DataSet named 'data'
data_v=loop_v.get_data_set(name="1D reflectometry sweep voltage fix")

#Plotting part
plot_v=qc.QtPlot(windowTitle="1D reflectometry sweep: gate voltage = "+str(fixed_volt)+" V")
plot_v.add(data_v.lockin_R)
plot_v.autorange()
loop_v.with_bg_task(plot_v.update)
loop_v.run()

# Saving 1D freq sweep in .csv
Using the same method as in 2D sweep

In [ ]:
#this is the dedicated variable for filepath
filepath_v=""
#first we convert the arrays into numpy arrays one by one
hameg_data_v=np.ravel(np.array(data_v.hameg_frequency_set))
lockin_data_v=np.ravel(np.array(data_v.lockin_R))
#then we use pandas to save the csv
df=pd.DataFrame(data=[hameg_data_v,lockin_data_v]).T
df.rename(columns={0:"Frequency (Hz)",1:"Reflected voltage (V)"},inplace=True)
df.to_csv(filepath_v+"1D_V_"+str(fixed_volt)+"_"+date_v+".csv",sep=",")
#print(df)
print("All data saved as csv!")

# Saving 1D frequency sweep in .png

In [ ]:
plot.save(filepath_v+"1D_V_"+str(fixed_volt)+"_"+date_v+".png")

# 1D sweep with fixed freq

In [3]:
#Fixed frequency of HAMEG
fixed_freq=1e6
hm.freq=fixed_freq

#Starting voltage in volts
start=-5
#End voltage in volts
end=5
#Step size in volts
delta=1
#Delay after each step in seconds
delay=1

#storing date and time
date_f=strftime("%y_%m_%d_%H_%M_%S")

loop_f=qc.Loop(gs.voltage.sweep(start,end,delta),delay=delay).each(sr.R)
data_f=loop_f.get_data_set(name="1D reflectometry sweep frequency fix")
plot_f=qc.QtPlot(windowTitle="1D reflectometry sweep: frequency = "+str(fixed_freq)+" Hz")
plot_f.add(data_f.lockin_R)
plot_f.autorange()
loop_f.with_bg_task(plot_f.update)
loop_f.run()

C:\Anaconda3\lib\site-packages\qcodes\plots\pyqtgraph.py:75: UserWarning: windowTitle argument has been changed to window_title. Please update your call to QtPlot
  warnings.warn("windowTitle argument has been changed to "


Started at 2018-09-05 18:26:57
DataSet:
   location = 'data/2018-09-05/#045_1D reflectometry sweep frequency fix_18-26-53'
   <Type>   | <array_id>       | <array.name> | <array.shape>
   Setpoint | yoko_voltage_set | voltage      | (11,)
   Measured | lockin_R         | R            | (11,)
Finished at 2018-09-05 18:27:08


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 11124: character maps to <undefined>

# Saving 1D frequency sweep in .csv
All the same as before..

In [4]:
#this is the dedicated variable for filepath
filepath_f=""
#first we convert the arrays into numpy arrays one by one
hameg_data_f=np.ravel(np.array(data_f.yoko_voltage_set))
lockin_data_f=np.ravel(np.array(data_f.lockin_R))
#then we use pandas to save the csv
df=pd.DataFrame(data=[hameg_data_f,lockin_data_f]).T
df.rename(columns={0:"Frequency (Hz)",1:"Reflected voltage (V)"},inplace=True)
df.to_csv(filepath_f+"1D_F_"+str(fixed_freq)+"_"+date_f+".csv",sep=",")
#print(df)
print("All data saved as csv!")

All data saved as csv!


# Saving 1D voltage sweep in .png

In [10]:
plot_f.save(filepath_f+"1D_F_"+str(fixed_freq)+"_"+date_f+".png")

# Some other irrelevant stuff. Might be useful for testing though

In [ ]:
#just testing the commands here...
hm.frequency.set(8)
print(hm.frequency.get())
print(sr.x.get())
gs.voltage.set(1)
print(gs.voltage.get())

In [ ]:
#we are sweeping freq and measuring sr.x value
#for sweep:(start,end,step) delay in secs
loop=qc.Loop(hm.frequency.sweep(100,1000,10),delay=0.01).each(sr.x)
data = loop.get_data_set(name='testsweep')

In [ ]:
print(data)

In [ ]:
plot = qc.MatPlot(data.lock_x)  # create a 1D plot
loop.with_bg_task(plot.update)
loop.run();

Here we try 2D loop and plot

In [ ]:
loop2=qc.Loop(hm.frequency.sweep(100,1000,10),delay=0.1).loop(gs.voltage.sweep(0,5,0.1),delay=0.1).each(sr.x)
data2 = loop.get_data_set(name='2D_test')

In [ ]:
plot2 = qc.MatPlot(data2.lock_x)
loop.with_bg_task(plot.update)
loop.run();